In [32]:
import os
import random
import math

import numpy as np
import pandas as pd

### camera name

In [34]:
bounds = [(95, 25), (35, -35), (-25, -95), (155, 85), (-145, 145), (-85, -155)]
bounds_coef = [(math.tan(math.pi*(y1/180)), math.tan(math.pi*(y2/180))) for (y1, y2) in bounds]
color_list = ['b', 'g', 'orange', 'c', 'm', 'y', 'k', 'w', 'r']
camera_name = ['front_left', 'front', 'front_right', 'back_left', 'back', 'back_right']
bounds_quat = [[3,1], [1,4], [4,3], [2,1], [3,2], [4,3]]

In [35]:
def check_cam_index(index, camera_index):
    points = [(anno.iloc[index, 4+i], anno.iloc[index, 8+i]) for i in range(4)]
#     print(np.sign(points))
    for x, y in points:
        
        if np.sign(x) == 1 and np.sign(y) == 1:
            quat = 1
        elif np.sign(x) == 1 and np.sign(y) == -1:
            quat = 4
        elif np.sign(x) == -1 and np.sign(y) == 1:
            quat = 2
        else:
            quat = 3
            
        if quat not in bounds_quat[camera_index]:
            continue
            
        angle = math.atan(y/x)/math.pi*180
        
        if quat in [1,2] and angle < 0:
            angle += 180
        elif quat in [3,4] and angle > 0:
            angle -= 180
        
        if camera_index == 4:
            if abs(angle) > 145:
                return True
        else:
            if angle >= min(bounds[camera_index]) and angle <= max(bounds[camera_index]):
                return True
            
    
    return False

In [36]:
def check_cam_row(row, camera_index):
    points = [(row.iloc[4+i], row.iloc[8+i]) for i in range(4)]
    for x, y in points:
        
        if np.sign(x) == 1 and np.sign(y) == 1:
            quat = 1
        elif np.sign(x) == 1 and np.sign(y) == -1:
            quat = 4
        elif np.sign(x) == -1 and np.sign(y) == 1:
            quat = 2
        else:
            quat = 3
            
        if quat not in bounds_quat[camera_index]:
            continue
            
        angle = math.atan(y/x)/math.pi*180
        
        if quat in [1,2] and angle < 0:
            angle += 180
        elif quat in [3,4] and angle > 0:
            angle -= 180
        
        if camera_index == 4:
            if abs(angle) > 145:
                return True
        else:
            if angle >= min(bounds[camera_index]) and angle <= max(bounds[camera_index]):
                return True
            
    
    return False

### distance

In [37]:
def get_distance(row):
    x = np.mean(row.iloc[4:8])
    y = np.mean(row.iloc[8:12])
    return np.sqrt(x**2 + y**2)

### weight/height/centerx/centery style annotation

In [26]:
def get_whxy(row):
    x = np.mean(row.iloc[4:8])
    y = np.mean(row.iloc[8:12])
    w = abs(max(row.iloc[4:8]) - min(row.iloc[4:8]))
    h = abs(max(row.iloc[8:12]) - min(row.iloc[8:12]))
    return np.array([x, y, w, h])

### Generate features and save file

In [42]:
def generate_features(anno_path,output_path):
    anno = pd.read_csv(anno_path)
    for index, name in enumerate(camera_name):
        anno[name] = anno.apply(lambda x: check_cam_row(x, index), axis = 1)
    anno['distance'] = anno.apply(lambda x: get_distance(x), axis = 1)
    anno['center_x'], anno['center_y'], anno['box_width'], anno['box_height'] = np.array(anno.apply(lambda x: get_whxy(x), axis = 1).tolist()).T
    anno.to_csv(output_path)
 

In [43]:
generate_features('data/annotation.csv','data/annotation_newfeat.csv')